Aqui importamos os módulos os e shutil, que são utilizados para manipular ficheiros e caminhos/diretórios.

Defenimos tamabém os caminhos das pastas, onde estão contidos os conjuntos/set's de treino, validação e teste.

In [2]:
import os, shutil
train_dir = '../train'
validation_dir = '../validation'
test_dir = '../test'

No seguinte código, começamos por definir a variável IMG_SIZE. Defenimos como 150x150 pois se a definíssemos com valor 32x32 (tamanho das imagens do dataset) os feature maps tornavam-se demasiado pequenos ou mesmo inválidos

Criamos os conjuntos de dados de treino, validação e teste a partir dos respetivos diretórios. Assim, é especificado o caminho do diretório para cada um dos conjuntos anteriormente relatados.

O label_mode foi definido como categorical, tendo em conta que as labels são vetores one-hot encoded.

O resante código esta implementado com um unico objetivo, dividir o dataset em 3 partes. Primeiro utilizamos uma função que devlve o tamanho do dataset e usamo-la para ir buscar o tamanho, depois com a funcao split dividimos o dataset, o que ela recebe é um dataset que queremos dividir e em quantas partes, neste caso em 1/3 do tamanho total.

In [7]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 150

train_dataset = image_dataset_from_directory(train_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))
validation_dataset = image_dataset_from_directory(validation_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))
test_dataset = image_dataset_from_directory(test_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))

#### Dividir o dataset ####
def get_dataset_size(dataset):
    return sum(1 for _ in dataset)

train_size = get_dataset_size(train_dataset)
validation_size = get_dataset_size(validation_dataset)

part_train_size = train_size // 5
part_validation_size = validation_size // 5

def split_dataset(dataset, part_size):
    parts = []
    for i in range(5):
        parts.append(dataset.skip(i * part_size).take(part_size))
    return parts

train_parts = split_dataset(train_dataset, part_train_size)
validation_parts = split_dataset(validation_dataset, part_validation_size)

train_dataset_1, train_dataset_2, train_dataset_3, train_dataset_4, train_dataset_5 = train_parts
validation_dataset_1, validation_dataset_2, validation_dataset_3, validation_dataset_4, validation_dataset_5 = validation_parts

print(f"Train dataset parts sizes: {[get_dataset_size(part) for part in train_parts]}")
print(f"Validation dataset parts sizes: {[get_dataset_size(part) for part in validation_parts]}")


Found 40000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Train dataset parts sizes: [416, 416, 416]
Validation dataset parts sizes: [104, 104, 104]


O código importa a arquitetura da rede convolucional VGG19 pré-treinada através do módulo tensorflow.keras. A VGG19 é uma arquitetura de rede neural convolucional (CNN) renomada, conhecida por sua profundidade e eficácia em tarefas de reconhecimento de imagens. 

weights='imagenet', indica que os pesos da rede foram inicializados com os valores aprendidos a partir do conjunto de dados ImageNet. 

include_top=False, configura a VGG19 para não incluir a fully connected layer no topo da rede. Isto permite-nos que os recursos extraídos pela VGG19 sejam utilizados como entrada para camadas posteriormente defenidas por nós, adequadas para uma tarefa específica, neste caso iremos usar o modelo pré treinado no ficheiro ModelT_transferLearning_featureExtraction_WithoutDataAumentation_OnlyClassification.h5 para classificação. A razão pela qual decidimos utilizar a estrategia referida anteriormente é porque para realizar o fine tuning devemos seguir os seguintes passos:
1. Adicionar a nossa rede personalizada em cima de uma rede base já treinada
2. Congelar a rede base (vgg19)
3. Treinar a parte que adicionámos(rede personalizada)
4. Descongelar algumas camadas da rede base (vgg19)
5. Treinar os dois modelos em conjunto o que adicionámos(rede personalizada) e o modelo base(vgg19)

Ao realizarmos esta estrategia é devido ao facto que podemos logo saltar para o ponto 3, reutilizando trabalho realizado.

input_shape=(IMG_SIZE, IMG_SIZE, 3) especifica o formato esperado para as imagens de entrada na rede.

In [8]:
from keras.applications.vgg19 import VGG19

conv_base = VGG19(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)) 
conv_base.trainable = True

# Deixar todas as camadas, exceto as últimas quatro, não treináveis (congeladas)
for layer in conv_base.layers[:-5]: 
    layer.trainable = False

Verificar as camadas da vgg19, no output conseguimes observar que o block5 está todo defenido como Trainable=True, o que fará com que as camadas do respetivo bloco atualizem os seus pesos ao longo do treino

In [10]:
from tensorflow.keras.models import Model

# Função para imprimir o estado das camadas de forma recursiva
def print_layer_trainable_status(layer, indent=0):
    print(f"{' ' * indent}Layer: {layer.name}, Trainable: {layer.trainable}")
    if isinstance(layer, Model):
        for sub_layer in layer.layers:
            print_layer_trainable_status(sub_layer, indent + 2)

print_layer_trainable_status(conv_base)

Layer: vgg19, Trainable: True
  Layer: input_1, Trainable: False
  Layer: block1_conv1, Trainable: False
  Layer: block1_conv2, Trainable: False
  Layer: block1_pool, Trainable: False
  Layer: block2_conv1, Trainable: False
  Layer: block2_conv2, Trainable: False
  Layer: block2_pool, Trainable: False
  Layer: block3_conv1, Trainable: False
  Layer: block3_conv2, Trainable: False
  Layer: block3_conv3, Trainable: False
  Layer: block3_conv4, Trainable: False
  Layer: block3_pool, Trainable: False
  Layer: block4_conv1, Trainable: False
  Layer: block4_conv2, Trainable: False
  Layer: block4_conv3, Trainable: False
  Layer: block4_conv4, Trainable: False
  Layer: block4_pool, Trainable: False
  Layer: block5_conv1, Trainable: True
  Layer: block5_conv2, Trainable: True
  Layer: block5_conv3, Trainable: True
  Layer: block5_conv4, Trainable: True
  Layer: block5_pool, Trainable: True


Fazer import do modelo que irá ser usado para classificação, e ver a sua arquitetura

In [11]:
from tensorflow import keras

#Adicionar o modelo treinado com feature extraction
base_model = keras.models.load_model('models/ModelT_transferLearning_featureExtraction_WithoutDataAumentation_OnlyClassification.h5')

base_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 4, 4, 512)]       0         
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               4194816   
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                           

No seguinte bloco de código é onde defenimos a layer para o data augmentation e fazemos a junção da mesma com os dois modelos, da CNN(Modelo usado para reconhecimento de imagens) e da NN(Modelo usado para classificação das imagens)

In [18]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import GaussianNoise

# Definir a sequência de data augmentation
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        GaussianNoise(stddev=0.1),  # Adicionar ruído gaussiano
    ]
)

# Criar uma camada de input com o mesmo shape da VGG19
input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Aplicar data augmentation à camada de input
augmented_input = data_augmentation(input_layer)

# Passar a camada de entrada aumentada para o modelo base VGG19
vgg19_output = conv_base(augmented_input)

# Passar a saída da VGG19 para o modelo pré-treinado
model_output = base_model(vgg19_output)

# Criar o novo modelo combinado com a VGG19 e o modelo pré-treinado
model = Model(inputs=input_layer, outputs=model_output)

Verificar a arquitetura do modelo completo

In [19]:
print_layer_trainable_status(model)

Layer: model, Trainable: True
  Layer: input_3, Trainable: True
  Layer: sequential_1, Trainable: True
    Layer: random_flip_1, Trainable: True
    Layer: random_rotation_1, Trainable: True
    Layer: random_zoom_1, Trainable: True
    Layer: random_contrast_1, Trainable: True
    Layer: gaussian_noise_1, Trainable: True
  Layer: vgg19, Trainable: True
    Layer: input_1, Trainable: False
    Layer: block1_conv1, Trainable: False
    Layer: block1_conv2, Trainable: False
    Layer: block1_pool, Trainable: False
    Layer: block2_conv1, Trainable: False
    Layer: block2_conv2, Trainable: False
    Layer: block2_pool, Trainable: False
    Layer: block3_conv1, Trainable: False
    Layer: block3_conv2, Trainable: False
    Layer: block3_conv3, Trainable: False
    Layer: block3_conv4, Trainable: False
    Layer: block3_pool, Trainable: False
    Layer: block4_conv1, Trainable: False
    Layer: block4_conv2, Trainable: False
    Layer: block4_conv3, Trainable: False
    Layer: block4_conv

In [20]:
#A partir deste bloco iremos treinar o modelo para os sub datasets
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint_callback = ModelCheckpoint(
    filepath='models/ModelT_transferLearning_fineTuning_WithDataAumentation_best.h5',
    save_best_only=True,
    monitor='val_loss',
    verbose=1 
)

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

In [22]:
#Subset 1
history = model.fit(train_dataset_1, epochs=10, validation_data=validation_dataset_1, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

Epoch 1/10


  3/416 [..............................] - ETA: 20:58 - loss: 1.9240 - accuracy: 0.5000

KeyboardInterrupt: 